In [1]:
import pandas as pd
import csv
import json
from functools import reduce

def deep_get(dictionary, keys, default=None):
    return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)

def convert_event(event):
    suffix = event['name']
    col_name = f"events.{suffix}"
    
    return ({
        f"{col_name}.value": event.get("value"),
        f"{col_name}.date.start": deep_get(event, "dateRange.start.$date"),
        f"{col_name}.date.end": deep_get(event, "dateRange.end.$date")
    })

def convert_string_list(item):
    try:
        if (type(item) != str) or (item == ''):
            return None
        if type(json.loads(item)) == list:
            return ", ".join(json.loads(item))
        else:
            return item
    except:
        print(item)
        
def process_chunk(infile, outfile=None):
    if not outfile:
        outfile = infile[:-4] + "_processed.csv"
    with open(outfile, 'w+') as g:
        with open(infile, 'r') as f:
            reader = csv.DictReader(f)
            writer = csv.DictWriter(g, fieldnames=fields, extrasaction='ignore')
            writer.writeheader()
            for row in reader:
                if "ObjectId" not in row['_id']:
                    continue
                if type(row['notes']) == str:
                    row['notes'] = row['notes'].replace('\n', ', ')
                row['notes']
                for event in json.loads(row['events']):
                    row = row | convert_event(event)
                writer.writerow(row)
    print(outfile)

In [2]:
df = pd.read_csv('../gh_test.csv', engine='python', error_bad_lines=False)

Skipping line 19: unexpected end of data


In [3]:
cols_to_add = []
for col_name in ["confirmed", "firstClinicalConsultation", "hospitalAdmission", "icuAdmission", "onsetSymptoms", "outcome", "selfIsolation"]:
    cols_to_add.append(f"events.{col_name}.value")
    cols_to_add.append(f"events.{col_name}.date.start")
    cols_to_add.append(f"events.{col_name}.date.end")

fields = set(df.columns).union(set(cols_to_add))
fields = sorted(list(fields - set(['events'])))

In [4]:
list_fields = ['caseReference.uploadIds',
'demographics.nationalities',
'symptoms.values',
'preexistingConditions.values',
'transmission.linkedCaseIds',
'transmission.places',
'transmission.routes',
'travelHistory.travel.methods']

In [5]:
import os

In [8]:
process_list = []
for file in os.listdir('../test/'):
    if file.startswith("gh"):
        process_list.append('../test/' + file)

In [9]:
for f in process_list:
    process_chunk(f)

../test/gh_0_processed.csv
../test/gh_1_processed.csv
../test/gh_10_processed.csv
../test/gh_11_processed.csv
../test/gh_12_processed.csv
../test/gh_13_processed.csv
../test/gh_14_processed.csv
../test/gh_15_processed.csv
../test/gh_16_processed.csv
../test/gh_17_processed.csv
../test/gh_18_processed.csv
../test/gh_19_processed.csv
../test/gh_2_processed.csv
../test/gh_20_processed.csv
../test/gh_21_processed.csv
../test/gh_22_processed.csv
../test/gh_23_processed.csv
../test/gh_24_processed.csv
../test/gh_25_processed.csv
../test/gh_26_processed.csv
../test/gh_27_processed.csv
../test/gh_28_processed.csv
../test/gh_29_processed.csv
../test/gh_3_processed.csv
../test/gh_30_processed.csv
../test/gh_31_processed.csv
../test/gh_32_processed.csv
../test/gh_33_processed.csv
../test/gh_34_processed.csv
../test/gh_35_processed.csv
../test/gh_36_processed.csv
../test/gh_37_processed.csv
../test/gh_38_processed.csv
../test/gh_39_processed.csv
../test/gh_4_processed.csv
../test/gh_40_processed.c

In [27]:
import shutil

In [28]:
for file in os.listdir('../'):
    if file.endswith("processed.csv"):
        shutil.move('../' + file, '../combine/' + file)